In [ ]:
from langchain_openai import ChatOpenAI
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""

In [2]:
llm = ChatOpenAI(model="gpt-4o")

In [3]:
output = llm.invoke("HI")
output

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-8afd948d-0d93-4d4b-86ce-190a56afb9e3-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 간단한 형태의 챗봇 구현

### 1. Output Parser 알아보기

- LLM이 생성한 텍스트를 정형화된 형태로 추출하는 법을 배웁니다.

### 2. Memory 추가 하기

- 단발성이 아닌, 과거 히스토리를 기억한 상태에서 텍스트를 생성하는 법을 배웁니다.

### 3. Chat message 깊이 파보기

- Memory 기능을 가능하게 하는 LangChain의 주요 기능을 깊이 살펴봅니다.

### 4. 간단한 챗봇 구현

- 지금까지 배운 prompt, llm, 그리고 memory를 모두 결합하여 간단한 챗봇을 만듭니다.

---

## 1. Output Parser 알아보기

반환 받은 응답에 대한 컨트롤을 잘하기 위해 output parser를 사용한다.

특정 형식(객체)를 정의하고 객체 필드를 구성, 필드의 유형을 강제하여 
일관된 결과값을 유도한다. 

pydantic 객체를 이용해서 output parser 객체를 정의, 
한번 더 output parser 를 llm 을 통해 fix 하는 과정을 거칠 수 있다. 

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# template를 정의
template = "{movie_name}에 나오는 등장인물을 알려줘."

# from_template : Langchain의 promptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

# LCEL을 활용하여 chain 객체 생성
chain = prompt | llm | StrOutputParser()

res = chain.invoke({"movie_name":"명탐정코난: 천국으로의 카운트다운"})
print(res)

"명탐정 코난: 천국으로의 카운트다운"은 명탐정 코난 시리즈의 극장판 중 하나로, 여러 주요 캐릭터들이 등장합니다. 주요 등장인물을 몇 명 소개하면 다음과 같습니다:

1. **에도가와 코난 (신이치 쿠도)** - 주인공인 어린 탐정.
2. **모리 란** - 코난의 친구로, 신이치의 소꿉친구.
3. **모리 코고로** - 란의 아버지이며 탐정.
4. **하이바라 아이 (미야노 시호)** - 검은 조직에서 탈출한 인물로, 코난과 협력하여 사건을 해결함.
5. **코지마 겐타, 츠부라야 미츠히코, 요시다 아유미** - 코난의 반 친구들이자 어린 탐정단.
6. **아가사 히로시** - 코난을 돕는 과학자 겸 발명가.
7. **검은 조직** - 주요 악역들로, 이 극장판에서는 특히 진(Gin)과 보드카(Vodka)가 중점적으로 등장함.

이 외에도, 극장판에 따라 다양한 부가적인 캐릭터가 등장하며, 이들이 사건의 중요한 단서를 제공하거나 사건의 배후에 있기도 합니다.


In [25]:
chain = prompt | llm

res = chain.invoke({"movie_name":"명탐정코난: 천국으로의 카운트다운"})
print(res)

content='영화 "명탐정 코난: 천국으로의 카운트다운"에는 다양한 등장인물이 나옵니다. 주요 등장인물은 다음과 같습니다.\n\n1. **코난 에도가와** - 본명 쿠도 신이치로, 고등학생 탐정이지만 약을 먹고 어린아이의 몸으로 변해버린 주인공.\n2. **모리 란** - 코난의 소꿉친구이자 쿠도 신이치의 여자친구. 가라데 유단자.\n3. **모리 코고로** - 란의 아버지로, 사립탐정. 코난의 추리 덕분에 유명해진 탐정.\n4. **하이바라 아이** - 과거 검은 조직의 일원이었지만 현재는 코난과 함께 검은 조직에 맞서 싸우는 동료.\n5. **검은 조직** - 신이치를 어린아이로 만든 범죄 조직. 주요 인물로는 진과 워카가 있다.\n\n이 외에도 탐정단 친구들과 경찰 인물들, 사건과 관련된 여러 등장인물이 영화에 출연합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 253, 'prompt_tokens': 31, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None} id='run-ccd5e3f8-f671-4d60-a27e-2a96c624a659-0' usage_metadata={'input_tokens': 31, 'output_tokens': 253, 'total_tokens': 

### typing
python 의 타입 hinting 을 지원하여 변수가 가질 자료형을 명시하여 가독성 높임

### pydantic
python 데이터 검증 및 설정 관리 지원

`BaseModel`: 해당 클래스를 상속받아 커스터마이징된 데이터 유효성 검증 모델 생성 
- 데이터를 특정 구조로 강제하여 더 쉽게 데이터 제어
- 직렬화 또는 역직렬화 지원하여 python 객체를 json 형식으로 쉽게 관리하기 때문

`Field` : Pydantic 데이터 모델에서 필드의 메타데이터를 정의할 때 사용
메타데이터란, 설명, 기본값, 값 범위등을 제공할 수 있어서 데이터 검증 시 세밀한 제어 가능

### pydaticOutputParser
- llm 출력(즉 응답 데이터)를 pydantic model로 변환하는데 사용되는 클래스

In [5]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

In [18]:
### [충격] 각 필드(변수)가 어떤 정보를 담도록 파싱되어야 하는지는, description으로 정의됨.
### llm이 description을 이해해서 자신의 답변을 특정 변수(필드)로 형식화 해준다는거임..!!!! 진짜 충격이다.. 

class Food(BaseModel): # BaseModel 상속받아 옴
    
    # name이라는 변수는 string 타입으로 한정 (타입 힌트를 통해 타입을 강제) --> 추후 데이터 자료형 유효성 검사 시 사용
    name: str = Field(description="name of a food")
    
    ingredients: List[str] = Field(description="list of names of the ingredients mentioned")
    # Field() 를 통해 필드의 메타데이터를 정의 / 필드란 클래스 안의 변수를 말함. 

food_query = "What are the ingredients used for making pizza?"

# output parser 객체 생성
parser = PydanticOutputParser(pydantic_object=Food)
# llm이 생성한 응답을 pydantic model 사용해서 구조화
# pydantic model = Food

In [19]:
from langchain.output_parsers import OutputFixingParser
# OutputFixingParser: 잘못된 출력 형식이 반환될 경우에 llm을 사용하여 자동으로 수정ㅁ ㅣㅊ 보정해주는 파서

# 최종적인 파서 객체 생성
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
chain = llm | parser
output = chain.invoke(food_query)

output

In [22]:
chain = llm | new_parser
output = chain.invoke(food_query)

In [23]:
output

Food(name='Pizza', ingredients=['Flour', 'Water', 'Yeast', 'Salt', 'Olive oil', 'Sugar', 'Tomatoes', 'Garlic', 'Herbs', 'Mozzarella', 'Parmesan', 'Provolone', 'Vegetables', 'Pepperoni', 'Sausage', 'Ham', 'Bacon', 'Anchovies', 'Pineapple', 'Arugula'])

In [41]:
    
class Character(BaseModel): # BaseModel 상속받아 옴
    name: str = Field(description="name of a movie character")
    age: str = Field(description="an age of a movie character")
    gender: str = Field(description="gender of a movie character")
    
    role: str = Field(description="role and characteristic of a movie character in the movie")

class Movie_Characters(BaseModel): 
    characters: List[Character] = Field(description="Character Model 의 리스트")

mparser = PydanticOutputParser(pydantic_object=Movie_Characters)
new_mparser = OutputFixingParser.from_llm(parser=mparser, llm=llm)

mchain = prompt | llm | new_mparser
output = mchain.invoke({"movie_name":"명탐정코난: 천국으로의 카운트다운"})

for c in output.characters:
    print(c)

name='에도가와 코난 (신이치)' age='17' gender='Male' role='주인공이자 명탐정'
name='모리 란' age='17' gender='Female' role='코난의 친구이자 신이치의 소꿉친구'
name='모리 코고로' age='38' gender='Male' role='유명한 탐정, 란의 아버지'
name='하이버라 아이' age='18' gender='Female' role='코난과 함께 검은 조직에 맞서는 인물'
name='검은 조직 인물들 (진, 보드카 등)' age='Unknown' gender='Mixed' role='코난에게 큰 위협이 되는 범죄 조직'


## 2. Memory 추가 하기

In [26]:
from langchain.memory import ConversationBufferMemory

- `save_context`를 활용하여 history를 추가할 수 있습니다.

In [27]:
memory = ConversationBufferMemory()
memory.save_context(
    inputs={
        "user": "내가 영화를 보고 싶은데 어떤걸 봐야 할지 모르겠어."
    },
    outputs={
        "assistant": "안녕하세요, 영화 추천이라고 하면 제가 전문이죠! 어떤 장르의 영화를 보고싶으신가요?"
    },
)

C:\Users\User\AppData\Local\Temp\ipykernel_14388\3471961393.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [28]:
memory.load_memory_variables({})

{'history': 'Human: 내가 영화를 보고 싶은데 어떤걸 봐야 할지 모르겠어.\nAI: 안녕하세요, 영화 추천이라고 하면 제가 전문이죠! 어떤 장르의 영화를 보고싶으신가요?'}

In [29]:
memory.save_context(
    inputs={"human": "액션 영화를 보고싶어. 판타지가 조금 가미된."},
    outputs={
        "ai": "네, 그렇다면 반지의 제왕을 추천합니다!"
    },
)

In [30]:
memory.load_memory_variables({})

{'history': 'Human: 내가 영화를 보고 싶은데 어떤걸 봐야 할지 모르겠어.\nAI: 안녕하세요, 영화 추천이라고 하면 제가 전문이죠! 어떤 장르의 영화를 보고싶으신가요?\nHuman: 액션 영화를 보고싶어. 판타지가 조금 가미된.\nAI: 네, 그렇다면 반지의 제왕을 추천합니다!'}

- `clear`를 활용하여 현재까지의 'history'를 삭제할 수 있습니다.

In [31]:
# memory.clear()

m_m = ConversationBufferMemory(return_messages = True)

m_m.save_context(
    inputs={"human": " 안녕 ~~~~~~~ "},
    outputs={
        "ai" : "안녕 !! ㅎㅎ 반가워"
    },
)

m_m.save_context(
    inputs={"human": "오늘 출근할까, 말까"},
    outputs={
        "ai" : "출근하기 싫으면 하지 말자!"
    },
)

m_m.load_memory_variables({}) # 메세지 객체로 history 존재

{'history': [HumanMessage(content=' 안녕 ~~~~~~~ ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕 !! ㅎㅎ 반가워', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘 출근할까, 말까', additional_kwargs={}, response_metadata={}),
  AIMessage(content='출근하기 싫으면 하지 말자!', additional_kwargs={}, response_metadata={})]}

## 3. Chat message 깊이 파보기

- <span style="background-color: #A5B68D; border-radius: 5px; padding: 2px 6px; border: 1px solid #ccc; font-family: sans-serif;">
    HumanMessage</span> : 사용자 메세지를 담은 객체입니다.
- <span style="background-color: #A5B68D; border-radius: 5px; padding: 2px 6px; border: 1px solid #ccc; font-family: sans-serif;">
    AIMessage</span> : LLM 메세지를 담은 객체입니다.

In [6]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="안녕?")])

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 10, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-718b64c9-911b-477b-b3bc-4e9b3043ca1d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage(content="안녕?"),
        AIMessage(content="안녕하세요. 오늘은 어떻게 도와드릴까요?"),
        HumanMessage(content="내 영어 이름을 만들어줘"),
    ]
)

AIMessage(content='물론이죠! 영어 이름을 정할 때는 발음, 의미, 그리고 개인의 취향을 고려하는 것이 좋습니다. 몇 가지 예시를 드릴 테니 마음에 드는 것이 있는지 확인해 보세요:\n\n1. **Sophia** (소피아) - 지혜롭다는 의미가 있습니다.\n2. **Ethan** (이든) - 강하고 견고한 의미를 가집니다.\n3. **Emma** (엠마) - 전통적이고 인기 있는 이름입니다.\n4. **Liam** (리엄) - 보호자를 뜻합니다.\n5. **Olivia** (올리비아) - 평화로운 의미와 관련이 있습니다.\n6. **Noah** (노아) - 안락함을 상징합니다.\n\n마음에 드는 이름이 있거나, 특정 의미를 가진 이름을 원하신다면 말씀해 주세요. 추가적인 제안도 가능합니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 36, 'total_tokens': 237, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-888f2e44-0c73-4ac8-a3dc-e88ae69b5db1-0', usage_metadata={'input_tokens': 36, 'output_tokens': 201, 'total_tokens': 237, 'input_

<span style="background-color: #A5B68D; border-radius: 5px; padding: 2px 6px; border: 1px solid #ccc; font-family: sans-serif;">
    Runnable</span>이란, langchain의 컴포넌트를 실행하고 연결할 수 있도록 하는 주요 객체입니다.

이전 `component1 | component2 | component3` 와 같은 chain이 가능한 이유이기도 합니다.

그 중 `RunnableWithMessageHistory`은 다른 runnable을 감싸어 chat message history를 읽고 업데이트 할 수 있도록 합니다.

In [10]:
from langchain_core.chat_history import (
    BaseChatMessageHistory, # 대화 히스토리를 관리하는 기본 클래스
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# session_id를 key로 삼아 dictionary에 저장
# value 값에는 chat history를 저장
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store: # 세션아이디 정보 없다 = 대화 히스토리가 store에 없다면
        store[session_id] = InMemoryChatMessageHistory() # 해당 객체 생성해서 현재 대화쌍을 sotre 에저장
    return store[session_id]

# get_session_history를 통해 우리가 관리하는 
# 대화 기록을 자동으로 처리해주는 러너블 객체 생성 = 러너블 객체의 실행 기능 + 히스토리 모듈의 대화 기록 관리 기능 
with_message_history = RunnableWithMessageHistory(llm, get_session_history)

Runnable에 메세지와 memory를 위한 configuration 값들을 제공합니다.

In [11]:
config = {"configurable": {"session_id": "movie123"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕, 내 이름은 엘리스야.")],
    config=config, # 동적으로 속성 제어
)
# session id=movie123 인 응답이 생성됨 

response.content

'안녕하세요, 엘리스! 만나서 반가워요. 어떻게 도와드릴까요?'

In [9]:
## runnable 객체가 대화 기록을 저장함. (질문msg 와 응답 모두 ..)
store

{'movie123': InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕, 내 이름은 엘리스야.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 엘리스! 만나서 반가워요. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 18, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-e1993e25-b872-4e79-bde9-41c9bf0b6d6a-0', usage_metadata={'input_tokens': 18, 'output_tokens': 23, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])}

In [10]:
config = {"configurable": {"session_id": "movie123"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐라고 했는지 기억나?")],
    config=config,
)

response.content

'네, 당신의 이름은 엘리스라고 말씀하셨어요.还有其他需要帮助的吗?'

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="왜 중국말 문장이 나왔는지 그 이유나 배경에 대해 알려줘.")],
    config=config,
)

response.content

'정확한 이유는 알 수 없으나, 제공된 답변이 의도와 다르게 중국어로 출력되는 경우는 드물지만 발생할 수 있는 오류입니다. 이는 알고리즘이나 데이터 처리 과정에서의 작은 혼동이나 기술적 오류로 발생했을 가능성이 있습니다. 불편을 드려 죄송하며, 최대한 빠르게 다시 올바른 정보를 제공하기 위해 노력하겠습니다.'

In [15]:
print(store)

{'movie123': InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕, 내 이름은 엘리스야.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 엘리스! 만나서 반가워요. 오늘 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 18, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-e1993e25-b872-4e79-bde9-41c9bf0b6d6a-0', usage_metadata={'input_tokens': 18, 'output_tokens': 23, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='내 이름이 뭐라고 했는지 기억나?', additional_kwargs={}, response_metadata={}), AIMessage(content='네,

In [12]:
### store에 저장된 메세지 히스토리 출력
for m in store['movie123'].messages:
    print(f">> {m.content}")

>> 안녕, 내 이름은 엘리스야.
>> 안녕하세요, 엘리스! 만나서 반가워요. 어떻게 도와드릴까요?


config 변경을 통해 상호작용할 memory를 변경할 수 있습니다.

In [ ]:
config = {"configurable": {"session_id": "random123"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐라고 했는지 기억나?")],
    config=config,
)

response.content

In [ ]:
store

## 4. 간단한 챗봇 구현

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

`MessagesPlaceholder`를 활용하여 prompt template에 message history가 들어갈 위치를 미리 지정할 수 있습니다.

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "당신은 {movie_name}에 관해서만 이야기를 하고 싶어하는 영화 전문가 입니다.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chat_chain = prompt | llm

In [14]:
store = {}

# session_id를 key로 삼아 dictionary에 저장함. value 값에는 chat history를 저장함
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_session_history,
    input_messages_key="messages",
)

chat_chain 자체가 runnable이기 때문에 `RunnableWithMessageHistory`를 활용하여 memory를 연동한 runnable로 구성할 수 있습니다.

In [15]:
config = {"configurable": {"session_id": "movie_chat"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="안녕, 분노의 질주에 누가 나오는지 알려줄 수 있어?")], "movie_name": "반지의 제왕"},
    config=config,
)

response.content

'안녕하세요! 저는 반지의 제왕에 대해 이야기하는 것을 더 좋아하지만, 분노의 질주 시리즈에 대해서는 아주 간단히 말씀드릴 수 있습니다. 분노의 질주 시리즈에는 빈 디젤, 폴 워커, 미셸 로드리게즈, 조던나 브루스터, 타이리스 깁슨, 루다크리스 등의 배우들이 출연합니다. \n\n하지만 반지의 제왕에 관심이 있으시면, 그 시리즈의 멋진 이야깃거리나 등장인물에 대해서 더욱 깊이 이야기 나눌 수 있습니다! 예를 들어, 엘리야 우드가 연기한 프로도 배긴스와 그가 펼치는 모험에 대해서 알고 싶으신가요?'

따라서 invoke를 할 때 'message'나 'movie_name'과 같은 변수를 input에 추가하여 chat_chain의 인풋으로 제공할 수 있습니다.

In [16]:

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="그렇다면 반지의 제왕은 총 몇 부작이야?")], "movie_name": "반지의 제왕"},
    config=config,
)

response.content

'반지의 제왕은 총 세 편의 영화로 이루어져 있습니다. 이 시리즈는 J.R.R. 톨킨의 소설을 바탕으로 하며, 피터 잭슨이 감독했습니다. 세 편의 영화는 다음과 같습니다:\n\n1. **반지의 제왕: 반지 원정대** (The Lord of the Rings: The Fellowship of the Ring, 2001)\n2. **반지의 제왕: 두 개의 탑** (The Lord of the Rings: The Two Towers, 2002)\n3. **반지의 제왕: 왕의 귀환** (The Lord of the Rings: The Return of the King, 2003)\n\n이 세 편의 영화는 스토리가 서로 연결되어 있으며, 직접적으로 이어지는 서사를 가지고 있습니다. 혹시 시리즈의 특정 부분이나 캐릭터에 대해 더 알고 싶으신 것이 있나요?'